In [1]:
# initialize modules and globals
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import os
import shutil
driver = webdriver.Chrome()

In [6]:
filetype = 'dropout'
local_html = './data/' + filetype + '/html/html.htm'
yearID = 'ctl00_ContentPlaceHolder1_ddYear'
gradeID = 'ctl00_ContentPlaceHolder1_ddGrade'
groupID = 'ctl00_ContentPlaceHolder1_ddSubgroup'
exportName = 'ctl00_ContentPlaceHolder1_spExport'
rawfileName = 'dropout.xlsx'

In [7]:
# load html to data
with open(local_html) as html:
    soup = BeautifulSoup(html, 'html.parser')

# scrape available years, grades, and student groups
years = []

for item in soup.find(id=yearID).children:
    if item.string != '\n':
        years.append(item.string)

print(years)

# grades = []

# for item in soup.find(id=gradeID).children:
#     if item.string != '\n':
#         grades.append(item.string)
        
# print(grades)        

groups = []

for item in soup.find(id=groupID).children:
    if item.string != '\n':
        groups.append(item.string)

print(groups)

['2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12', '2010-11', '2009-10', '2008-09', '2007-08']
['All Students', 'Female', 'Male', 'Economically Disadvantaged', 'High Needs', 'English Learner', 'Low Income', 'Students with disabilities', 'African American/Black', 'American Indian or Alaskan Native', 'Asian', 'Hispanic or Latino', 'Multi-race, non-Hispanic or Latino', 'Native Hawaiian or Pacific Islander', 'White']


In [9]:
# loop through all combinations of year/grade/group, downloading each file and renaming it
# recreate html page, with each option selected
for year in years:
#     for grade in grades:
    for group in groups:
        for item in soup.find(id=yearID).children:
            try:
                if item.string == year:
                    item['selected'] = 'selected'
                if item.string != year:
                    del item['selected']
            except TypeError:
                pass
#     for item in soup.find(id=gradeID).children:
#         try:
#             if item.string == grade:
#                 item['selected'] = 'selected'
#             if item.string != grade:
#                 del item['selected']
#         except TypeError:
#             pass
        for item in soup.find(id=groupID).children:
            try:
                if item.string == group:
                    item['selected'] = 'selected'
                if item.string != group:
                    del item['selected']
            except TypeError:
                pass
        html_file = open('./data/' + filetype + '/html/temp.html','w')
        html_file.write(soup.prettify())
        html_file.close()
        driver.get('file:///'+os.getcwd()+'/' + '/data/' + filetype + '/html/temp.html')

        # simulate export click with selenium
        download_button = driver.find_element_by_id(exportName)
        download_button.click()

        # wait for file to download
        time.sleep(5)

        # rename file
    #     filename = 'mcas_' + grade.replace('.', '').replace('/', '') + '_' + group.replace('.', '').replace('/', '') + '_' + year.replace('.', '').replace('/', '') + '.xlsx'
        filename = 'ma_'+ filetype + '_' + group.replace('.', '').replace('/', '') + '_' + year.replace('.', '').replace('/', '') + '.xlsx'
        os.rename('/Users/esegr1/Downloads/' + rawfileName,'/Users/esegr1/Downloads/' + filename)
        shutil.move(os.path.join('/Users/esegr1/Downloads/',filename), os.path.join('/Users/esegr1/Documents/repos/mcpsa/data/' + filetype + '/raw', filename))